In [ ]:
# Install project dependencies 
# %pip install python-dotenv
# %pip install openai
# %pip install langchain
# %pip install google-search-results
# %pip install deeplake tiktoken


In [ ]:
import os
from dotenv import load_dotenv
import textwrap
# Set the maximum line width
max_line_width = 80

# Load environment variables from .env file
load_dotenv()
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
embeddings = OpenAIEmbeddings()

# Access the variables
openai_api_key = os.getenv('OPENAI_API_KEY')
activeloop_api_key = os.getenv('ACTIVELOOP_API_KEY')
# !activeloop login -t 


In [ ]:
# Imports
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

In [ ]:
# Initialize OpenAI wrapper
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [ ]:
print(prompt.format(product="colorful socks"))

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt, )

In [ ]:
chain.run("colorful socks")

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
# tools = load_tools(["serpapi", "llm-math"], llm=llm)


# # Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# # Now let's test it out!
# agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
# Load all files from a directory
from langchain.document_loaders import TextLoader

root_dir = './clone-nayms'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [ ]:
# Chunk the documents into smaller pieces
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [ ]:
db = DeepLake.from_documents(texts, embeddings)

In [ ]:
import deeplake
ds = deeplake.load('mem://langchain')
# ds = deeplake.load('hub://davitbun/twitter-algorithm')
ds.visualize()

In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 20

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

# model = ChatOpenAI(model='gpt-4') # 'gpt-3.5-turbo',
# model = ChatOpenAI(model='gpt-3.5-turbo') # 'gpt-3.5-turbo',
model = ChatOpenAI(temperature=0) # 'gpt-3.5-turbo',
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [ ]:
def ask_questions(qa, questions, chat_history=None, max_line_width=80):
    # Initialize chat_history if it's not provided
    if chat_history is None:
        chat_history = []
    # Iterate over the questions
    for question in questions:  
        # Process the question using the given qa object (ConversationalRetrievalChain)
        result = qa({"question": question, "chat_history": chat_history})
        # Append the question-answer pair to chat_history
        chat_history.append((question, result['answer']))

        # Wrap the text
        wrapped_answer = textwrap.fill(result['answer'], width=max_line_width)
 
        # Print the question
        print(f"-> **Question**: {question}\n")
        
        # Print the wrapped answer
        print(f"**Answer**:\n{wrapped_answer}\n")
    # Return the wrapped answer text and updated chat_history
    # return wrapped_answer, chat_history

In [ ]:
questions = [
    "What does AppStorage do?",
] 

ask_questions(qa, questions, None)

In [ ]:
# Usage example:
questions = [
    "Give me a list of all of the methods in AdminFacet.",
] 

ask_questions(qa, questions)

In [ ]:
questions = [
    "What does withdrawDividend() do and how does it work? Can you show me an example of where it has a bug?",
] 

ask_questions(qa, questions)

In [ ]:
questions = [
    "Are dividends correctly calculated?",
] 

ask_questions(qa, questions)

In [ ]:
questions = [
    "How does the access control in Nayms work? The logic for the Nayms access control is called ACL and the logic is in the ACLFacet.sol file.",
] 

ask_questions(qa, questions)